# AutoML для временных рядов с FEDOT Industrial

В этом ноутбуке мы тестируем российскую AutoML библиотеку **FEDOT Industrial** для прогнозирования финансовых временных рядов с использованием walk-forward валидации.

## Особенности FEDOT Industrial:
- **AutoML подход**: автоматический выбор архитектуры модели
- **Композитные модели**: может строить сложные пайплайны из нескольких алгоритмов
- **Специализация на временных рядах**: оптимизированные алгоритмы для TS
- **Эволюционная оптимизация**: автоматический поиск лучших гиперпараметров
- **Российская разработка**: NCCR, ITMO University

## Преимущества AutoML:
- Не требует экспертных знаний в ML
- Автоматически находит лучшую архитектуру
- Оптимизирует гиперпараметры
- Строит комплексные пайплайны

## Методология:
- **Walk-forward валидация**: модель переобучается после каждого прогноза
- **Разбивка данных**: 95% обучение, 5% тест
- **Горизонт прогноза**: 1 шаг вперед
- **AutoML**: FEDOT сам выберет лучшую модель


In [ ]:
!pip install virtualenv

In [ ]:
import virtualenv
import subprocess
import os
import sys

notebook_name = "fedot"
virtualenv.cli_run(["venvs/" + notebook_name, "--no-download"])

venv_dir = "venvs/" + notebook_name
python_path = os.path.join(venv_dir, "bin", "python")
display_name = "Python (" + notebook_name + ")"
kernel_name = notebook_name

subprocess.check_call([os.path.join(venv_dir, "bin", "pip"), "install", "ipykernel"])

subprocess.check_call([
    python_path, "-m", "ipykernel", "install",
    "--user",
    "--name", kernel_name,
    "--display-name", display_name
])

In [1]:
import sys
print(sys.executable)

/workspace/predictors/univariate/venvs/fedot/bin/python


In [ ]:
import subprocess
import os
import sys

pip_path = os.path.join(sys.prefix, "bin", "pip")

subprocess.check_call([pip_path, "install", "fedot[extra]", "opencv-python-headless", "matplotlib", "scikit-learn", "pandas", "numpy", "seaborn",])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# FEDOT
from fedot import Fedot
from fedot.core.data.data import InputData
from fedot.core.data.data_split import train_test_data_setup
from fedot.core.repository.dataset_types import DataTypesEnum
from fedot.core.repository.tasks import Task, TaskTypesEnum, TsForecastingParams

pd.options.display.float_format = '{:.4f}'.format

print("FEDOT загружен успешно!")

FEDOT загружен успешно!


In [ ]:
def directional_accuracy(actual: np.ndarray, predicted: np.ndarray) -> float:
    """
    Доля совпавших направлений изменения: рост/падение.
    """
    actual_dir = np.sign(np.diff(actual))
    pred_dir   = np.sign(np.diff(predicted))
    
    return (actual_dir == pred_dir).mean() * 100

# Загрузка данных

In [ ]:
def load_series_map(series_dir="../../data/series"):
    """Загружает временные ряды из CSV файлов"""
    series_map = {}
    series_path = Path(series_dir)

    if not series_path.exists():
        raise FileNotFoundError(f"Папка не найдена: {series_dir}")
    
    for csv_file in series_path.glob("*.csv"):
        ticker = csv_file.stem.upper()
        df = pd.read_csv(csv_file)
        if "timestamp" not in df.columns or "close" not in df.columns:
            raise ValueError(f"{csv_file.name} не содержит 'timestamp' или 'close'")
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
        df = df[["timestamp", "close"]].sort_values("timestamp").reset_index(drop=True)
        series_map[ticker] = df
    
    return series_map

series_map = load_series_map("../../data/series")
print(f"Загружено {len(series_map)} временных рядов:")
for ticker, df in series_map.items():
    print(f"  {ticker}: {len(df)} точек данных")

Загружено 10 временных рядов:
  AFLT: 2375 точек данных
  LKOH: 2375 точек данных
  MOEX: 2375 точек данных
  NVTK: 2373 точек данных
  PIKK: 2375 точек данных
  SBER: 2375 точек данных
  VKCO: 1197 точек данных
  VTBR: 1722 точек данных
  X5: 1499 точек данных
  YDEX: 2339 точек данных


In [ ]:
def prepare_fedot_data(series_map, test_ratio=0.05):
    """
    Подготавливает данные для FEDOT с разбивкой 95%/5%
    
    Args:
        series_map: словарь с временными рядами
        test_ratio: доля тестовых данных (0.05 = 5%)
    
    Returns:
        prepared_data: словарь с подготовленными данными для каждого тикера
    """
    prepared_data = {}
    
    for ticker, df in series_map.items():
        values = df['close'].values
        timestamps = df['timestamp'].values
        
        split_idx = int(len(values) * (1 - test_ratio))
        train_data = values[:split_idx]
        test_data = values[split_idx:]
        train_timestamps = timestamps[:split_idx]
        test_timestamps = timestamps[split_idx:]
        
        prepared_data[ticker] = {
            'full_data': values,
            'train_data': train_data,
            'test_data': test_data,
            'train_timestamps': train_timestamps,
            'test_timestamps': test_timestamps,
            'split_idx': split_idx
        }
        
        print(f"{ticker}: train={len(train_data)}, test={len(test_data)}")
    
    return prepared_data

def create_fedot_input_data(data, forecast_length=1):
    """
    Создает InputData для FEDOT из временного ряда
    
    Args:
        data: массив значений временного ряда
        forecast_length: горизонт прогноза
    
    Returns:
        InputData: объект данных для FEDOT
    """
    task = Task(
        TaskTypesEnum.ts_forecasting,
        TsForecastingParams(forecast_length=forecast_length)
    )
    
    # Создаем InputData
    return InputData.from_numpy_time_series(
        features_array=ts_values,
        target_array=ts_values,
        idx=ts_timestamps,
        task=task,
        data_type=DataTypesEnum.ts
    )

prepared_data = prepare_fedot_data(series_map, test_ratio=0.05)
print(f"\nДанные подготовлены для {len(prepared_data)} тикеров")

AFLT: train=2256, test=119
LKOH: train=2256, test=119
MOEX: train=2256, test=119
NVTK: train=2254, test=119
PIKK: train=2256, test=119
SBER: train=2256, test=119
VKCO: train=1137, test=60
VTBR: train=1635, test=87
X5: train=1424, test=75
YDEX: train=2222, test=117

Данные подготовлены для 10 тикеров


In [ ]:
def create_fedot_input_data(values: np.ndarray,
                            timestamps: np.ndarray,
                            forecast_length: int = 1) -> InputData:
    """Фабрика InputData для временных рядов."""
    task = Task(
        TaskTypesEnum.ts_forecasting,
        TsForecastingParams(forecast_length=forecast_length)
    )
    return InputData.from_numpy_time_series(
        features_array=values,
        target_array=values,
        idx=timestamps,
        task=task,
        data_type=DataTypesEnum.ts
    )

In [ ]:
def walk_forward_fedot(ticker: str,
                       ticker_data: dict,
                       timeout_minutes: float = 5.0):
    """
    Однократное обучение FEDOT + walk-forward прогнозы.
    Время = train_time + avg(prediction_time).
    """
    print(f"\n=== {ticker}: FEDOT AutoML fast Walk-forward ===")

    train_vals = ticker_data['train_data']
    test_vals  = ticker_data['test_data']
    train_idx  = ticker_data['train_timestamps']
    test_idx   = ticker_data['test_timestamps']
    split_idx  = ticker_data['split_idx']

    input_train = create_fedot_input_data(
        values=train_vals,
        timestamps=train_idx,
        forecast_length=1
    )
    auto_model = Fedot(
        problem='ts_forecasting',
        timeout=timeout_minutes,
        preset='ts',
        n_jobs=1,
        safe_mode=True,
        cv_folds=3,
        early_stopping_iterations=10,
        with_tuning=False
    )
    t0 = time.time()
    auto_model.fit(features=input_train, target=train_vals)
    train_time = time.time() - t0
    print(f"  Trained in {train_time:.2f}s → pipeline: {auto_model.current_pipeline}")

    preds = []
    times = []
    full_vals = np.concatenate([train_vals, test_vals])
    full_idx  = np.concatenate([train_idx, test_idx])
    n_steps   = len(test_vals)

    for i in range(n_steps):
        end = split_idx + i
        hist_vals = full_vals[:end]
        hist_idx  = full_idx[:end]

        if len(hist_vals) < 10:
            continue

        input_loop = create_fedot_input_data(
            values=hist_vals,
            timestamps=hist_idx,
            forecast_length=1
        )
        train_data, forecast_data = train_test_data_setup(input_loop)

        t1 = time.time()
        raw_forecast = auto_model.forecast(forecast_data)
        dt = time.time() - t1

        if hasattr(raw_forecast, 'predict'):
            pred = raw_forecast.predict[0]
        elif isinstance(raw_forecast, np.ndarray):
            pred = raw_forecast[0]
        elif isinstance(raw_forecast, list):
            pred = raw_forecast[0]
        else:
            pred = float(raw_forecast)

        preds.append(pred)
        times.append(dt)

    preds = np.array(preds)
    actuals = test_vals[:len(preds)]

    mae_v   = mean_absolute_error(actuals, preds)
    rmse_v  = mean_squared_error(actuals, preds, squared=False)
    mape_v  = mean_absolute_percentage_error(actuals, preds) * 100
    da_v    = directional_accuracy(actuals, preds)

    avg_pred_time = np.mean(times)
    eff_time_step = train_time + avg_pred_time

    print(f"\nResults for {ticker}:")
    print(f"  MAE:      {mae_v:.4f}")
    print(f"  MAPE:     {mape_v:.2f}%")
    print(f"  RMSE:     {rmse_v:.4f}")
    print(f"  DA:       {da_v:.2f}%")
    print(f"  Train:    {train_time:.2f}s")
    print(f"  Avg pred: {avg_pred_time:.4f}s")
    print(f"  Eff/step: {eff_time_step:.4f}s")

    return {
        'actuals': test_vals,
        'predictions': preds,
        'times':       times,
        'metrics': {
            'MAE':               mae_v,
            'MAPE (%)':          mape_v,
            'RMSE':              rmse_v,
            'DA (%)':            da_v,
            'Train Time (s)':    train_time,
            'Avg Pred Time (s)': avg_pred_time,
            'Eff Time/step (s)': eff_time_step
        }
    }

In [ ]:
all_results = {}

test_tickers = list(prepared_data.keys())
print(f"Тестируем FEDOT AutoML на тикерах: {test_tickers}")
print(f"Это может занять некоторое время, так как FEDOT ищет оптимальную архитектуру...")

fedot_configs = {
    'FEDOT_Fast': {
        'timeout_minutes': 1,  # Быстрый режим
        'description': 'Быстрый поиск модели (1 мин)'
    },
    'FEDOT_Balanced': {
        'timeout_minutes': 5,  # Сбалансированный режим  
        'description': 'Сбалансированный поиск (5 мин)'
    }
}

for ticker in test_tickers:
    print(f"\n{'='*80}")
    print(f"ОБРАБОТКА ТИКЕРА: {ticker}")
    print(f"{'='*80}")
    
    ticker_data = prepared_data[ticker]
    ticker_results = {}
    
    for config_name, config in fedot_configs.items():
        try:
            print(f"\n--- Конфигурация: {config_name} ({config['description']}) ---")
            
            result = walk_forward_fedot(
                ticker=ticker,
                ticker_data=ticker_data,
                timeout_minutes=config['timeout_minutes']
            )
            
            ticker_results[config_name] = result
            
        except Exception as e:
            print(f"\n!!! ОШИБКА с конфигурацией {config_name} для {ticker}: {e}")
            ticker_results[config_name] = {
                'metrics': {
                    'MAE': np.nan,
                    'MAPE (%)': np.nan,
                    'RMSE': np.nan,
                    'DA (%)': np.nan,
                    'Avg Time (s)': np.nan
                },
                'model_analysis': {}
            }

            
    
    all_results[ticker] = ticker_results

print(f"\n{'='*80}")
print("FEDOT AUTOML ОБРАБОТКА ЗАВЕРШЕНА")
print(f"{'='*80}")

Тестируем FEDOT AutoML на тикерах: ['AFLT', 'LKOH', 'MOEX', 'NVTK', 'PIKK', 'SBER', 'VKCO', 'VTBR', 'X5', 'YDEX']
Это может занять некоторое время, так как FEDOT ищет оптимальную архитектуру...

ОБРАБОТКА ТИКЕРА: AFLT

--- Конфигурация: FEDOT_Fast (Быстрый поиск модели (1 мин)) ---

=== AFLT: FEDOT AutoML fast Walk-forward ===
2025-06-17 18:26:16,302 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 111
2025-06-17 18:26:16,456 - ApiComposer - Initial pipeline was fitted in 0.2 sec.
2025-06-17 18:26:16,459 - ApiComposer - Taking into account n_folds=3, estimated fit time for initial assumption is 0.5 sec.
2025-06-17 18:26:16,476 - ApiComposer - AutoML configured. Parameters tuning: False. Time limit: 1 min. Set of candidate models: ['adareg', 'ar', 'diff_filter', 'dtreg', 'ets', 'fast_ica', 'gaussian_filter', 'glm', 'lagged', 'lasso', 'linear', 'locf', 'normalization', 'pca', 'polyfit', 'ridge', 'scaling', 'sgdr', 'sm

Generations:   0%|          | 0/10000 [00:00<?, ?gen/s]

2025-06-17 18:26:18,434 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 292
2025-06-17 18:26:20,846 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 292
2025-06-17 18:26:22,134 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 292
2025-06-17 18:26:34,103 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 292
2025-06-17 18:26:35,318 - MultiprocessingDispatcher - 4 individuals out of 4 in previous population were evaluated successfully.
2025-06-17 18:26:47,337 - SparseLaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 292
2025-06-17 18:26:51,213 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSize

Generations:   0%|          | 0/10000 [01:52<?, ?gen/s]

2025-06-17 18:28:09,521 - ApiComposer - Model generation finished


...

2025-06-17 18:40:13,891 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 216
2025-06-17 18:40:16,128 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 216
2025-06-17 18:40:16,838 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 216
2025-06-17 18:40:23,610 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 216
2025-06-17 18:40:24,854 - MultiprocessingDispatcher - 4 individuals out of 4 in previous population were evaluated successfully.
2025-06-17 18:42:15,700 - GroupedCondition - Optimisation stopped: Time limit is reached


Generations:   0%|          | 0/10000 [02:03<?, ?gen/s]


2025-06-17 18:42:15,932 - ApiComposer - Model generation finished
2025-06-17 18:42:16,042 - FEDOT logger - Final pipeline was fitted
2025-06-17 18:42:16,044 - FEDOT logger - Final pipeline: {'depth': 2, 'length': 2, 'nodes': [ar, smoothing]}
ar - {'lag_1': 7, 'lag_2': 12}
smoothing - {}
  Trained in 124.51s → pipeline: {'depth': 2, 'length': 2, 'nodes': [ar, smoothing]}

Results for LKOH:
  MAE:      262.7068
  MAPE:     3.76%
  RMSE:     336.4433
  DA:       48.31%
  Train:    124.51s
  Avg pred: 0.0143s
  Eff/step: 124.5242s

--- Конфигурация: FEDOT_Balanced (Сбалансированный поиск (5 мин)) ---

=== LKOH: FEDOT AutoML fast Walk-forward ===
2025-06-17 18:42:18,074 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 556
2025-06-17 18:42:18,287 - ApiComposer - Initial pipeline was fitted in 0.2 sec.
2025-06-17 18:42:18,289 - ApiComposer - Taking into account n_folds=3, estimated fit time for initial assumption is 0.7 se

Generations:   0%|          | 0/10000 [00:00<?, ?gen/s]

2025-06-17 18:42:20,668 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 216
2025-06-17 18:42:23,375 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 216
2025-06-17 18:42:24,226 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 216
2025-06-17 18:42:31,741 - LaggedTransformationImplementation - Window size of lagged transformation was changed by WindowSizeSelector from 0 to 216
2025-06-17 18:42:33,185 - MultiprocessingDispatcher - 4 individuals out of 4 in previous population were evaluated successfully.


In [ ]:
summary_data = []
model_analysis_data = []

for ticker, ticker_results in all_results.items():
    for config_name, result in ticker_results.items():
        metrics = result['metrics']
        summary_data.append({
            'Ticker': ticker,
            'Config': config_name,
            'MAE': metrics['MAE'],
            'MAPE (%)': metrics['MAPE (%)'],
            'RMSE': metrics['RMSE'],
            'DA (%)': metrics['DA (%)'],
            'Avg Time (s)': metrics['Eff Time/step (s)']
        })
        
        if 'model_analysis' in result:
            for model_name, count in result['model_analysis'].items():
                model_analysis_data.append({
                    'Ticker': ticker,
                    'Config': config_name,
                    'Model': model_name,
                    'Usage_Count': count
                })

summary_df = pd.DataFrame(summary_data)

model_df = pd.DataFrame(model_analysis_data)

if len(summary_df) > 0:
    config_summary = summary_df.groupby('Config').agg({
        'MAE': 'mean',
        'MAPE (%)': 'mean',
        'RMSE': 'mean',
        'DA (%)': 'mean',
        'Avg Time (s)': 'mean'
    }).round(4)
    
    print("\n" + "="*80)
    print("ИТОГОВАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ FEDOT AUTOML")
    print("="*80)
    print("\nДетальные результаты по тикерам и конфигурациям:")
    display(summary_df.round(4))
    
    print("\nСредние показатели по конфигурациям:")
    display(config_summary)
    
    if len(model_df) > 0:
        print("\nАнализ автоматически найденных моделей:")
        model_summary = model_df.groupby(['Config', 'Model']).agg({
            'Usage_Count': 'sum'
        }).reset_index().sort_values('Usage_Count', ascending=False)
        display(model_summary.head(10))

    if len(summary_df) > 1:
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('FEDOT AutoML - Walk-forward валидация', fontsize=16)
        
        summary_df_clean = summary_df.dropna()
        
        if len(summary_df_clean) > 0:
            pivot_mape = summary_df_clean.pivot(index='Ticker', columns='Config', values='MAPE (%)')
            if not pivot_mape.empty:
                pivot_mape.plot(kind='bar', ax=axes[0,0])
                axes[0,0].set_title('MAPE (%)')
                axes[0,0].set_ylabel('MAPE (%)')
                axes[0,0].legend()
            
            pivot_rmse = summary_df_clean.pivot(index='Ticker', columns='Config', values='RMSE')
            if not pivot_rmse.empty:
                pivot_rmse.plot(kind='bar', ax=axes[0,1])
                axes[0,1].set_title('RMSE')
                axes[0,1].set_ylabel('RMSE')
                axes[0,1].legend()
            
            pivot_da = summary_df_clean.pivot(index='Ticker', columns='Config', values='DA (%)')
            if not pivot_da.empty:
                pivot_da.plot(kind='bar', ax=axes[1,0])
                axes[1,0].set_title('Directional Accuracy (%)')
                axes[1,0].set_ylabel('DA (%)')
                axes[1,0].legend()
            
            pivot_time = summary_df_clean.pivot(index='Ticker', columns='Config', values='Avg Time (s)')
            if not pivot_time.empty:
                pivot_time.plot(kind='bar', ax=axes[1,1])
                axes[1,1].set_title('Среднее время поиска и прогноза (сек)')
                axes[1,1].set_ylabel('Время (с)')
                axes[1,1].legend()
        
        plt.tight_layout()
        plt.show()
        
        if len(model_df) > 0:
            plt.figure(figsize=(12, 6))
            model_counts = model_df.groupby('Model')['Usage_Count'].sum().sort_values(ascending=False)
            
            if len(model_counts) > 0:
                model_counts.head(10).plot(kind='bar')
                plt.title('Топ-10 моделей, найденных FEDOT AutoML')
                plt.xlabel('Модель')
                plt.ylabel('Количество использований')
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                plt.show()
else:
    print("Нет данных для отображения")

In [ ]:
def plot_fedot_forecasts(ticker, ticker_results, ticker_data):
    """Построение графиков прогнозов FEDOT для одного тикера"""
    
    best_config = None
    best_mape = float('inf')
    
    for config_name, result in ticker_results.items():
        if 'metrics' in result and not np.isnan(result['metrics']['MAPE (%)']):
            if result['metrics']['MAPE (%)'] < best_mape:
                best_mape = result['metrics']['MAPE (%)']
                best_config = config_name
    
    if best_config is None:
        print(f"Нет успешных результатов для {ticker}")
        return
    
    successful_configs = []
    for config_name, result in ticker_results.items():
        if 'predictions' in result and len(result['predictions']) > 0:
            successful_configs.append(config_name)
    
    if len(successful_configs) == 0:
        print(f"Нет данных для построения графиков для {ticker}")
        return
    
    n_configs = len(successful_configs)
    fig, axes = plt.subplots(1, n_configs, figsize=(15, 5))
    if n_configs == 1:
        axes = [axes]
    
    fig.suptitle(f'FEDOT AutoML Walk-forward прогнозы для {ticker}', fontsize=14)
    
    for i, config_name in enumerate(successful_configs):
        ax = axes[i]
        result = ticker_results[config_name]
        
        predictions = result['predictions']
        actuals = result['actuals']
        
        time_points = range(len(actuals))
        ax.plot(time_points, actuals, label='Реальные значения', color='blue', linewidth=2)
        ax.plot(time_points, predictions, label='Прогноз AutoML', color='red', linestyle='--', linewidth=2)
        
        metrics = result['metrics']
        title = f"{config_name}"
        if config_name == best_config:
            title += " (Лучшая)"
        title += f"\nMAPE: {metrics['MAPE (%)']:.2f}%, DA: {metrics['DA (%)']:.1f}%"
        
        ax.set_title(title)
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_xlabel('Шаг времени')
        ax.set_ylabel('Значение цены')
    
    plt.tight_layout()
    plt.show()
    
    if best_config and 'model_analysis' in ticker_results[best_config]:
        print(f"\nМодели, найденные лучшей конфигурацией {best_config}:")
        model_analysis = ticker_results[best_config]['model_analysis']
        for model_name, count in sorted(model_analysis.items(), key=lambda x: x[1], reverse=True):
            print(f"  {model_name}: {count} раз(а)")

print("\n" + "="*80)
print("ВИЗУАЛИЗАЦИЯ ПРОГНОЗОВ FEDOT AUTOML")
print("="*80)

for ticker in test_tickers:
    if ticker in all_results:
        plot_fedot_forecasts(ticker, all_results[ticker], prepared_data[ticker])